# Kaggle Titanic : Toujours plus loin !

In [1]:
import pandas as pd
import re
import numpy as np

from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

pd.options.display.max_columns = None
#pd.set_printoptions(max_columns=500)

TRAIN = pd.read_csv("../datasources/titanic/train.csv")
TEST = pd.read_csv("../datasources/titanic/test.csv")
FULL = pd.concat([TRAIN, TEST]) # Assemble les deux jeux de données

/opt/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



# Récupère la localisation de la cabine : babord ou tribord)
https://fr.wikipedia.org/wiki/Passerelle_de_navigation_du_Titanic

L'évacuation des passagers dans les canots est organisée ainsi : le premier officier William Murdoch est chargé de tous les canots situés à tribord (c'est-à-dire tous les canots portant un chiffre impair en plus des canots A et C) et le deuxième officier Charles Lightoller est chargé de tous les canots situés à bâbord (tous les canots portant un chiffre pair en plus des canots B et D). Les autres officiers se doivent d'assister Murdoch et Lightoller dans leurs tâches. À 0 h 55, le troisième officier Pitman aide au remplissage du canot no 5 puis embarque dans celui-ci pour en prendre le commandement.

Les stratégies ont cependant différé entre tribord et bâbord. Les différences principales se tiennent par l'interprétation de la règle des « femmes et des enfants d'abord » et par le nombre de personnes embarquées. D'abord, William Murdoch a fait embarquer en moyenne 42 personnes pour les dix canots dont il s'est occupé, contre 32 personnes en moyenne pour huit canots par Charles Lightoller. La méthode de Murdoch consistait à donner la priorité aux femmes et aux enfants et à faire monter les hommes pour remplir les places restées vides (par exemple le canot no 15 comportait environ 50 hommes pour 65 personnes au total). Au contraire, Lightoller ne faisait monter que des femmes et des enfants et interdisait quasiment systématiquement l'accès aux hommes, préférant retenir les canots en attendant que davantage de femmes et d'enfants ne se présentent. La méthode de Lightoller rendait difficile le plein chargement des canots car il n'y avait sur le Titanic que 500 femmes et enfants alors que les canots pouvaient accueillir au total 1 178 personnes. De même, Lightoller craignait de mettre en danger les canots si ceux-ci étaient trop chargés. La méthode de Murdoch a permis que les canots soient mieux remplis mais aussi qu'ils soient plus vite mis à la mer ; par exemple à 1 h 45, il ne restait que deux canots à tribord contre encore cinq à bâbord. On peut aussi constater que les trois derniers canots tribord ont été quasiment remplis au maximum alors que Lightoller n'en a rempli aucun en totalité. Enfin, aucun des deux hommes n'est parvenu à mettre à l'eau correctement les canots situés au-dessus du pont des officiers, du fait de leur emplacement éloigné des bossoirs et par manque de temps (source wikipédia).

# Création d'une fonction de préparation

In [2]:
###################################
# Création d'un DataFrame avec tous les noms de famille
###################################
def addFamilyName(data):
    fn = data.copy()
    fn['FamilyName'] = ''
    for idx, noms in enumerate(fn["Name"]):
        fn.loc[idx, 'FamilyName'] = re.search('([A-Za-z0-9]*),\ ([A-Za-z0-9 ]*)\. (.*)', noms).group(1)
    return fn

###################################
# Fonction retournant un DF avec tous les noms de familles du jeu de test (+train dans le décompte) avec le nombre de membre
###################################
def getFamilyNamesOnlyInTest():
    # Création d'un DataFrame avec tous les noms de famille (full)
    familynames = []
    for noms in FULL["Name"]:
        familynames.append(re.search('([A-Za-z0-9]*),\ ([A-Za-z0-9 ]*)\. (.*)', noms).group(1))
    pdfamilynames = pd.DataFrame(familynames, columns = ['FamilyName'])

    # Créé une liste filtrée des noms de famille avec plus de 2 occurences !
    famsurv = FULL.join(pdfamilynames)
    famCount = famsurv['FamilyName'].value_counts()
    pdfamCounts = pd.DataFrame(famCount, columns = ['FamilyName'])
    pdfamCounts['famCount'] = pdfamCounts['FamilyName']
    pdfamCounts['FamilyName'] = pdfamCounts.index
    pdfamCounts = pdfamCounts[pdfamCounts['famCount'] >= 2]

    # Liste des familles dans le jeu de test
    famtest = pd.DataFrame(addFamilyName(TEST)['FamilyName'].value_counts(), columns = ['FamilyName'])
    famtest['famCount'] = famtest['FamilyName']
    famtest['FamilyName'] = famtest.index
    del famtest['famCount']

    # filtre les noms de familles qui n'existent que dans le jeu de test
    return pd.merge(famtest, pdfamCounts, how='inner', on='FamilyName')

###################################
# fonction créant deux features SurnameOneLived et SurnameAllDied
###################################
def getFamilySurv(traindata):
    # Notion de survivant / famille
    tdata = addFamilyName(traindata)
    # Compte le nb de personne par famille
    sruvPerFam = pd.DataFrame(tdata.groupby(['FamilyName'])['Survived'].count())
    sruvPerFam['Count'] = sruvPerFam['Survived']
    del sruvPerFam['Survived']
    # Compte le nb de survivant par famille
    sruvPerFam = sruvPerFam.join(pd.DataFrame(tdata.groupby(['FamilyName'])['Survived'].sum()))
    sruvPerFam['NbSurv'] = sruvPerFam['Survived']
    del sruvPerFam['Survived']
    # retire les personnes seules
    sruvPerFam = sruvPerFam[sruvPerFam['Count'] > 1]
    sruvPerFam['FamilyName'] = sruvPerFam.index
    sruvPerFam['SurnameOneLived'] = 0 # Au moins un survit
    sruvPerFam['SurnameAllDied'] = 0 # Tous meurrent
    for row in range(sruvPerFam.shape[0]):
        if (sruvPerFam.iloc[row, 1] >= 1):
            sruvPerFam.iloc[row, 3] = 1
        elif sruvPerFam.iloc[row, 1] == 0:
            sruvPerFam.iloc[row, 4] = 1
    del sruvPerFam['NbSurv']
    del sruvPerFam['Count']
    return sruvPerFam

###################################
# Fonction ajoutant les colonnes dummy noms famille dans un DF
###################################
def addDummyFamilyName(data):
    # ajoute les colonnes nulles/dummies avec les noms de famille
    filtre = getFamilyNamesOnlyInTest()
    for family in filtre['FamilyName']:
        data[family] = 0
    # récupère le nom de famille dans le DF
    for idx, f in enumerate(data["Name"]):
        # Modifie les colonnes dummies du nom de famille en 1 ou 0 selon le nom de famille
        iNom = re.search('([A-Za-z0-9]*),\ ([A-Za-z0-9 ]*)\. (.*)', f).group(1)
        for col in data.columns:
            if (col == iNom):
                data.loc[idx, col] = 1

###################################
# Fonction globale de préparation : Retourne le DataFrame préparé
###################################
def featureEngineering(data):
    fulldim = data.copy()
    
    # Sexe
    sexe = pd.get_dummies(fulldim['Sex'], prefix='sex')
    fulldim = fulldim.join(sexe)

    # Cabine, récupération du pont (on remplace le pont T proche du pont A)
    pont = pd.get_dummies(fulldim['Cabin'].fillna('X').str[0].replace('T', 'A'), prefix='Cabin')
    fulldim = fulldim.join(pont)
    
    # Cabine à Babord ou Tribord ?
    fulldim['CabinNumber'] = 0
    fulldim['CabinBaTri'] = 0
    fulldim['Cabin'] = fulldim['Cabin'].fillna('X0')
    for idx, cab in enumerate(fulldim['Cabin']):
        cb = cab.split(" ")[0]
        if cb[1:].isnumeric():
            numcab = int(cb[1:])
        else:
            numcab = 0
        # Récupère le numéro de cabine
        fulldim.loc[idx, 'CabinNumber']= numcab
        # Affecte pair (0) ou Impair (1) / Babord ou Tribord
        if (numcab > 0): 
            fulldim.loc[idx, 'CabinBaTri'] = int(numcab % 2)
        else:
            fulldim.loc[idx, 'CabinBaTri'] = 2 # inconnu !
    
    # Titre et Rang
    fulldim['Titre'] = fulldim.Name.map(lambda x : x.split(",")[1].split(".")[0]).fillna('X')
    fulldim['Rang'] = 0
    vip = ['Don','Sir', 'Major', 'Col', 'Jonkheer', 'Dr']
    femmeenfant = ['Miss', 'Mrs', 'Lady', 'Mlle', 'the Countess', 'Ms', 'Mme', 'Dona', 'Master']
    for idx, titre in enumerate(fulldim['Titre']): 
        if (titre.strip() in femmeenfant) :
            fulldim.loc[idx, 'Rang'] = 'FE'
        elif (titre.strip() in vip) :
            fulldim.loc[idx, 'Rang'] = 'VIP'
        else :
            fulldim.loc[idx, 'Rang'] = 'Autres'
    rg = pd.get_dummies(fulldim['Rang'], prefix='Rang')
    fulldim = fulldim.join(rg)
    
    # Age et catégories d'age
    age = fulldim['Age'].fillna(fulldim['Age'].median()) # rempl. NaN par Age médian
    catAge = []
    for i in range(fulldim.shape[0]) :
        if age[i] <= 3:
            catAge.append("bebe")
        elif age[i] > 3 and age[i] <= 16:
            catAge.append("enfant")
        elif age[i] > 16 and age[i] < 60:
            catAge.append("adulte") 
        else:
            catAge.append("vieux")
    catage = pd.DataFrame(catAge, columns = ['catAge'])
    # Force la catégorie d'age pour les Master (jeunes hommes)
    for idx, titre in enumerate(fulldim['Titre']): 
        if titre.strip() == 'Master':
            catage.loc[idx, 'catAge'] = 'enfant'      
    dumAge = pd.get_dummies(pd.DataFrame(catage, columns = ['catAge']), prefix='catAge')
    fulldim = fulldim.join(dumAge)
    
    # Embarquement ! est-ce nécessaire ???
    emb = pd.get_dummies(fulldim['Embarked'], prefix='emb')
    #fulldim = fulldim.join(emb)
    
    # Prix unitaire - Ticket, Prépartion d'un DF (TicketCounts) contenant les ticket avec leur nb d'occurence
    TicketCounts = pd.DataFrame(fulldim['Ticket'].value_counts().head())
    TicketCounts['TicketCount'] = TicketCounts['Ticket'] # renomme la colonne Ticket
    TicketCounts['Ticket'] = TicketCounts.index # rajoute une colonne Ticket pour le merge (jointure)
    # reporte le résultat dans le dataframe test (jointure des datasets)
    fin = pd.merge(fulldim, TicketCounts, how='left', on='Ticket')
    fin['PrixUnitaire'] = fin['Fare'] / fin['TicketCount'].fillna(1)
    prxunit = pd.DataFrame(fin['PrixUnitaire'])
    # Prix moyen 3eme classe (pour le passager de 3eme qui n'a pas de prix) ... on aurait pu faire une fonction ici ;-)
    prx3eme = fulldim.loc[fulldim['Pclass'] == 3]['Fare'].mean()
    prxunit = prxunit['PrixUnitaire'].fillna(prx3eme)
    fulldim = fulldim.join(prxunit)
    
    # Classe
    #pc = pd.DataFrame(MinMaxScaler().fit_transform(data[['Pclass']]), columns = ['Classe'])
    pc = pd.get_dummies(fulldim['Pclass'], prefix='Classe')
    fulldim = fulldim.join(pc)
    
    # family count
    fulldim['familyCount'] = fulldim['SibSp'] + fulldim['Parch']
    
    # compte les survivants par familles
    fulldim = addFamilyName(fulldim)
    survPerFam = getFamilySurv(TRAIN)
    fulldim = pd.merge(fulldim, survPerFam, how='left', on='FamilyName')
    fulldim['SurnameOneLived'] = fulldim['SurnameOneLived'].fillna(0)
    fulldim['SurnameAllDied'] = fulldim['SurnameAllDied'].fillna(0)

    # Ajoute les colonnes dummies sur le nom de famille
    addDummyFamilyName(fulldim)

    # Supprime les colonnes d'origine et de travail (temporaires)
    columns = ['Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Pclass', 'Cabin', 'Embarked',
               'Titre', 'Rang', 'CabinNumber', 'FamilyName']
    fulldim.drop(columns, inplace=True, axis=1)
    
    return fulldim

In [3]:
Xtrain = featureEngineering(TRAIN)
del Xtrain['Survived']
Xtest = featureEngineering(TEST)
y = TRAIN.Survived

ValueError: 'FamilyName' is both an index level and a column label, which is ambiguous.

In [7]:
Xtrain.head()

PassengerId  sex_female  sex_male  Cabin_A  Cabin_B  Cabin_C  Cabin_D  \
0            1           0         1        0        0        0        0   
1            2           1         0        0        0        1        0   
2            3           1         0        0        0        0        0   
3            4           1         0        0        0        1        0   
4            5           0         1        0        0        0        0   

   Cabin_E  Cabin_F  Cabin_G  Cabin_X  CabinBaTri  Rang_Autres  Rang_FE  \
0        0        0        0        1           2            1        0   
1        0        0        0        0           1            0        1   
2        0        0        0        1           2            0        1   
3        0        0        0        0           1            0        1   
4        0        0        0        1           2            1        0   

   Rang_VIP  catAge_adulte  catAge_bebe  catAge_enfant  catAge_vieux  \
0         0              1            0              0             0   
1         0              1            0              0             0   
2         0              1            0              0             0   
3         0              1            0              0             0   
4         0              1            0              0             0   

   PrixUnitaire  Classe_1  Classe_2  Classe_3  familyCount  SurnameOneLived  \
0        7.2500         0         0         1            1              0.0   
1       71.2833         1         0         0            1              0.0   
2        7.9250         0         0         1            0              0.0   
3       53.1000         1         0         0            1              1.0   
4        8.0500         0         0         1            0              1.0   

   SurnameAllDied  Davies  Asplund  Sage  Ryerson  Widener  Fortune  Goodwin  \
0             1.0       0        0     0        0        0        0        0   
1             0.0       0        0     0        0        0        0        0   
2             0.0       0        0     0        0        0        0        0   
3             0.0       0        0     0        0        0        0        0   
4             0.0       0        0     0        0        0        0        0   

   Touma  Drew  Smith  Becker  Samaan  Ford  Hocking  Warren  Nilsson  \
0      0     0      0       0       0     0        0       0        0   
1      0     0      0       0       0     0        0       0        0   
2      0     0      0       0       0     0        0       0        0   
3      0     0      0       0       0     0        0       0        0   
4      0     0      0       0       0     0        0       0        0   

   Heilmann  Herman  Dean  Billiard  Johnston  Spedden  Andersson  Brown  \
0         0       0     0         0         0        0          0      0   
1         0       0     0         0         0        0          0      0   
2         0       0     0         0         0        0          0      0   
3         0       0     0         0         0        0          0      0   
4         0       0     0         0         0        0          0      0   

   Cacic  Klasen  Keane  Robins  Greenfield  Boulos  Nakid  Harder  Hays  \
0      0       0      0       0           0       0      0       0     0   
1      0       0      0       0           0       0      0       0     0   
2      0       0      0       0           0       0      0       0     0   
3      0       0      0       0           0       0      0       0     0   
4      0       0      0       0           0       0      0       0     0   

   Faunthorpe  Chronopoulos  Svensson  Cribb  Carlo  Johansson  Crosby  Fox  \
0           0             0         0      0      0          0       0    0   
1           0             0         0      0      0          0       0    0   
2           0             0         0      0      0          0       0    0   
3           0             0         0

# Entrainement du modèle sur un Gradient Boosting

In [ ]:
gb = GradientBoostingClassifier(n_estimators=600, learning_rate=0.5, max_depth=11, random_state=4)
gb.fit(Xtrain, y)
p_gb = gb.predict(Xtrain)
print ("Score Train -->", round(gb.score(Xtrain, y) *100,4), " %")

## Validation croisée

In [ ]:
scores = cross_val_score(gb, Xtrain, y, cv=5)
print ("Validation croisée : {}", format(scores))
print ("Moyenne : ", round(scores.mean() *100,4), " %")

## Recherche sur grille

In [ ]:
param_grid = { 'n_estimators' : [600, 700, 800], 
               'learning_rate' : [0.3, 0.4, 0.5],
               'max_depth' : [8, 10, 11],
               'random_state' : [4, 5]}
grid_search = GridSearchCV(GradientBoostingClassifier(), param_grid, cv=5)
grid_search.fit(Xtrain, y)
print ("Score final : ", round(grid_search.score(Xtrain, y) *100,4), " %")
print ("Meilleurs parametres: ", grid_search.best_params_)
print ("Meilleure config: ", grid_search.best_estimator_)

# Formattage des résultats pour Kaggle ;-)

In [ ]:
p_test = gb.predict(Xtest)
result = pd.DataFrame(test['PassengerId'])
pred = pd.DataFrame(p_test, columns=['Survived'])
result = result.join(pred)
result.describe()

In [ ]:
result.to_csv("./data/resultrgb6.csv", columns=["PassengerId", "Survived"], index=False)

# Entrainement du modèle sur un Random Forest

In [ ]:
rf = RandomForestClassifier(n_estimators=1000, random_state=3, max_features=1)
rf.fit(Xtrain, y)
p_tr = rf.predict(Xtrain)
print ("Score Train -->", round(rf.score(Xtrain, y) *100,4), " %")

## Recherche sur grille

In [ ]:
param_grid_rf = { 'n_estimators' : [800, 1000], 
               'max_features' : [1, 0.5, 0.2],
               'random_state' : [3, 4, 5]}
grid_search_rf = GridSearchCV(RandomForestClassifier(), param_grid_rf, cv=5)
grid_search_rf.fit(Xtrain, y)
print ("Score final : ", round(grid_search_rf.score(Xtrain, y) *100,4), " %")
print ("Meilleurs parametres: ", grid_search_rf.best_params_)
print ("Meilleure config: ", grid_search_rf.best_estimator_)

Le résultat est nettement moins bon que le Gradient Boosting !

# Score : 78,9%